In [4]:
# Cleaning buoy data, including removing rows with NAs, ensuring all longitude values are 0 to 360, removing rows with Lat values less than 66, and 
# removing files with less than 50 rows

# TO DO: remove data on land

import os
import pandas as pd
import geopandas as gpd

directory = '../data/raw/buoydata/past'

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Remove rows with NAs
        df_cleaned = df.dropna()

        # Ensure that all longitude values are 0 to 360 and not -180 to 180
        df_cleaned['Lon'] = df_cleaned['Lon'].apply(lambda x: x + 360 if x < 0 else x)

        # Remove rows with Lat values less than 66
        df_cleaned = df_cleaned[df_cleaned['Lat'] >= 66]

        # Save the cleaned data back to the file unless the file has less than 50 rows
        if len(df_cleaned) > 50:
            df_cleaned.to_csv(file_path, index=False)
        else:
            os.remove(file_path)

        # Print a message to indicate if a file was deleted for having less than 50 rows
        if len(df_cleaned) <= 50:
            print(f'{filename} has been deleted for having less than 50 rows')

# Print a message to indicate that the script has finished
print('All files have been cleaned')

145803.csv has had its Lat values changed
300025010734900.csv has been deleted for having less than 50 rows
300234062644380.csv has been deleted for having less than 50 rows
300234066034140.csv has been deleted for having less than 50 rows
300234066215880.csv has been deleted for having less than 50 rows
300234066215880.csv has had its Lat values changed
300234066216670.csv has been deleted for having less than 50 rows
300234066216670.csv has had its Lat values changed
300234066216690.csv has been deleted for having less than 50 rows
300234066216690.csv has had its Lat values changed
300234066216700.csv has been deleted for having less than 50 rows
300234067874480.csv has been deleted for having less than 50 rows
300234067877380.csv has been deleted for having less than 50 rows
300234067976260.csv has been deleted for having less than 50 rows
300234068040050.csv has had its Lat values changed
300234068044480.csv has had its Lat values changed
300234068045040.csv has had its Lat values 